# Verification Example 
This notebook contains a verification study to verify ROLLO's optimization capabilities. 
The Ackley function is a commonly used evolutionary algorithm single objective benchmark problem. 

## Ackley Function 
The [Ackley Function](https://www.sfu.ca/~ssurjano/ackley.html) is a non-convex 
function, commonly used as a performance test for single-objective optimization 
algorithms:

$f(x) = -a \cdot exp \left(-b\sqrt{\frac{1}{d}\Sigma_{i=1}^dx_i^2}\right) - exp \left(\frac{1}{d}\Sigma_{i=1}^d cos(cx_i)\right) + a + exp(1)$

The recommended variable values are a = 20, b = 0.2, and c = 2π. The Ackley function’s global minimum point is f (0, 0) = 0. The figure below shows the resulting two-variable Ackley function (taken from [here](https://www.sfu.ca/~ssurjano/ackley.html)):

![Ackley](ackley.png)

The ROLLO optimization simulation is succcessful if it gets close to finding the minimum point at f(0,0) = 0

## ROLLO Input File 

ROLLO varies `x1` and `x2` control variables to minimize the `ackley` output parameter. The ackley function calculation occurs in `ackley_calculation.py`

In [25]:
rollo_input_file = open("ackley_optimization.json", "w")
rollo_input_file.write("""{
        "control_variables": {
            "x1": {"min": -32.768, "max": 32.768},
            "x2": {"min": -32.768, "max": 32.768}
        },
        "evaluators": {
            "ackley_evaluator": {
                "order": 0,
                "input_script": ["python", "ackley_calculation.py"],
                "inputs": ["x1", "x2"],
                "output_script": ["python", "get_ackley_output.py"],
                "outputs": ["ackley"]
            }
        },
        "constraints": {},
        "algorithm": {
            "objective": ["min"],
            "weight": [1.0],
            "optimized_variable": ["ackley"],
            "pop_size": 100,
            "generations": 10,
            "parallel": "none",
            "keep_files": "none"
        }
    }
""")
rollo_input_file.close()

## ROLLO `ackley_calculation.py` input script to calculate ackley function value. 

In [26]:
ackley_calculation = open("ackley_calculation.py", "w")
ackley_calculation.write("""
import numpy as np

x1 = {{x1}}
x2 = {{x2}}
ackley = (
    -20 * np.exp(-0.2 * np.sqrt(1 / 2 * (x1 ** 2 + x2 ** 2)))
    - np.exp(1 / 2 * (np.cos(2 * np.pi * x1) + np.cos(2 * np.pi * x2)))
    + 20
    + np.exp(1)
)

print(ackley)
""")
ackley_calculation.close()

## ROLLO get_ackley_output.py to return ackley function value to ROLLO. 

In [27]:
ackley_output = open("get_ackley_output.py", "w")
ackley_output.write("""
import numpy as np
import ast

with open("ackley_evaluator_input_script_out.txt") as file:
    contents = file.read()
ackley = ast.literal_eval(contents)
print({"ackley": ackley})
""")
ackley_output.close()

## Run ROLLO Optimization 

In [28]:
import subprocess 

subprocess.call("python -m rollo -i ackley_optimization.json", shell=True)

Entering generation 0...
       	   	     	                             oup                             	                                       ind                                       
       	   	     	-------------------------------------------------------------	---------------------------------------------------------------------------------
time   	gen	evals	avg          	std        	min         	max          	avg                    	min                        	max                      
71.6808	0  	100  	[20.48285334]	[1.8043881]	[10.4943522]	[22.22686589]	[3.60978399 2.9847639 ]	[-29.61501785 -32.18171576]	[31.57073548 31.97002803]
Entering generation 1...
       	   	     	                             oup                             	                                       ind                                       
       	   	     	-------------------------------------------------------------	---------------------------------------------------------------------------------
time

0

## Analyze Results Checkpoint File 

In [58]:
import pickle 
import numpy as np
from deap import creator, base

creator.create("obj", base.Fitness, weights=(1.0,))
creator.create("Ind", list, fitness=creator.obj)

with open("checkpoint.pkl", "rb") as cp_file:
    cp = pickle.load(cp_file)

logbook = cp["all"]

/Users/gwenchee/mambaforge3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'obj' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/gwenchee/mambaforge3/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Ind' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [73]:
final_gen = logbook["populations"][-1]
final_gen_ackley_values = np.array([ind.output[0] for ind in final_gen])
final_gen_ackley_sort = np.argsort(final_gen_ackley_values)[0]

# final generation, individual with smallest ackley function value: 
print("Ackley Function Value:", final_gen_ackley_values[final_gen_ackley_sort], ', (x,y): ', final_gen[final_gen_ackley_sort])

Ackley Function Value: 0.054781859969839974 , (x,y):  [0.006417761456496023, -0.015457276149234152]


ROLLO successfully found the minimum point of the ackley function. 